# Getting tables from the database

This short tutorial explains how to retrieve full tables from the database into [pandas DataFrames](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html).

## The following table are available from ``mendeleev``

* elements
* ionicradii
* ionizationenergies
* oxidationstates
* groups
* series
* isotopes

``mendeleev`` provides a convenient function `get_table` to perform the task at hand. The function can be directly imported from `mendeleev`

In [2]:
from mendeleev import get_table

To retrieve a table call the ``get_table`` with the table name as argument. Here we'll get probably the most important table ``elements`` with basis data on each element

In [3]:
ptable = get_table('elements')

Now we can use [pandas'](http://pandas.pydata.org) capabilities to work with the data. 

In [4]:
ptable.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118 entries, 0 to 117
Data columns (total 46 columns):
annotation                  118 non-null object
atomic_number               118 non-null int64
atomic_radius               88 non-null float64
atomic_volume               91 non-null float64
block                       118 non-null object
boiling_point               96 non-null float64
density                     95 non-null float64
description                 109 non-null object
dipole_polarizability       106 non-null float64
electron_affinity           77 non-null float64
electronic_configuration    118 non-null object
evaporation_heat            88 non-null float64
fusion_heat                 75 non-null float64
group_id                    90 non-null float64
lattice_constant            87 non-null float64
lattice_structure           91 non-null object
mass                        113 non-null float64
melting_point               100 non-null float64
name                        11

For clarity let's take only a subset of columns 

In [5]:
cols = ['atomic_number', 'symbol', 'atomic_radius', 'en_pauling', 'mass', 'block', 'vdw_radius_mm3']

In [6]:
ptable[cols].head()

,atomic_number,symbol,atomic_radius,en_pauling,mass,block,vdw_radius_mm3
0,1,H,79,2.20,1.007940,s,162
1,2,He,NaN,NaN,4.002602,s,153
2,3,Li,155,0.98,6.941000,s,255
3,4,Be,112,1.57,9.012180,s,223
4,5,B,98,2.04,10.811000,p,215


It is quite easy now to get descriptive statistics on the data.

In [7]:
ptable[cols].describe()

,atomic_number,atomic_radius,en_pauling,mass,vdw_radius_mm3
count,118.000000,88.000000,85.000000,113.000000,94.000000
mean,59.500000,169.397727,1.748588,139.516306,248.468085
std,34.207699,49.810108,0.634442,85.270136,36.017828
min,1.000000,79.000000,0.700000,1.007940,153.000000
25%,30.250000,137.000000,1.240000,63.546000,229.000000
50%,59.500000,160.000000,1.700000,138.905500,244.000000
75%,88.750000,181.000000,2.160000,209.987100,269.250000
max,118.000000,299.000000,3.980000,289.000000,364.000000


## Isotopes table

Let try and retrieve another table, namely ``isotopes``

In [8]:
isotopes = get_table('isotopes', index_col='id')

In [9]:
isotopes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 313 entries, 1 to 313
Data columns (total 4 columns):
atomic_number    313 non-null int64
mass             313 non-null float64
abundance        313 non-null float64
mass_number      313 non-null int64
dtypes: float64(2), int64(2)
memory usage: 12.2 KB


### Merge the elements table with the isotopes

We can now perform SQL-like merge operation on two ``DataFrame``s and produce an [outer](http://pandas.pydata.org/pandas-docs/stable/merging.html#database-style-dataframe-joining-merging) join 

In [10]:
import pandas as pd

In [11]:
merged = pd.merge(ptable[cols], isotopes, how='outer', on='atomic_number')

now we have the following columns in the ``merged`` ``DataFrame``

In [12]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 322 entries, 0 to 321
Data columns (total 10 columns):
atomic_number     322 non-null int64
symbol            322 non-null object
atomic_radius     270 non-null float64
en_pauling        272 non-null float64
mass_x            317 non-null float64
block             322 non-null object
vdw_radius_mm3    298 non-null float64
mass_y            313 non-null float64
abundance         313 non-null float64
mass_number       313 non-null float64
dtypes: float64(7), int64(1), object(2)
memory usage: 27.7+ KB


In [13]:
merged.head()

,atomic_number,symbol,atomic_radius,en_pauling,mass_x,block,vdw_radius_mm3,mass_y,abundance,mass_number
0,1,H,79,2.20,1.007940,s,162,1.007825,0.999885,1
1,1,H,79,2.20,1.007940,s,162,2.014102,0.000115,2
2,2,He,NaN,NaN,4.002602,s,153,3.016029,0.000001,3
3,2,He,NaN,NaN,4.002602,s,153,4.002603,0.999999,4
4,3,Li,155,0.98,6.941000,s,255,6.015122,0.075900,6


To display all the isotopes of Silicon

In [14]:
merged[merged['symbol'] == 'Si']

,atomic_number,symbol,atomic_radius,en_pauling,mass_x,block,vdw_radius_mm3,mass_y,abundance,mass_number
25,14,Si,132,1.9,28.0855,p,229,27.976927,0.922297,28
26,14,Si,132,1.9,28.0855,p,229,28.976495,0.046832,29
27,14,Si,132,1.9,28.0855,p,229,29.973770,0.030871,30


In [16]:
%version_information mendeleev, numpy, scipy, pandas

Software versions
Python 3.4.2 64bit [GCC 4.9.1]
IPython 4.1.1
OS Linux 3.16.0 4 amd64 x86_64 with debian 8.3
mendeleev 0.2.4
numpy 1.10.4
scipy 0.17.0
pandas 0.17.1
wto lut 16 22:08:48 2016 CET